In [1]:
from interactovery.transcripts import Transcripts
from interactovery.openaiwrap import OpenAiWrap

from dotenv import load_dotenv
import os
from pathlib import Path

import logging.config

prj_root = os.getcwd() + '/../'
conf_dir = prj_root + 'conf'

logging.config.fileConfig(f"{conf_dir}/logging.conf")
log = logging.getLogger('mainLogger')

env_path = Path(f"{conf_dir}/dev.env")
load_dotenv(dotenv_path=env_path)

user_prompt_gen_transcript = """The transcript should be of an executive assistant receiving calls for 
their executive and needing to schedule a follow-up at a later date.  Randomly decide on a date and time and how many 
turns it takes to book the appointment.  Randomly decide on people names, but make sure to use diverse names.  Don't 
use the words "my executive" or "your executive" or similar expressions.  Randomly decide on the reason the executive 
is unavailable.  Don't use an executive service as a business."""

openai_api_key = os.getenv('OPENAI_API_KEY')
openai_org = os.getenv('OPENAI_ORG')
openai_client = OpenAiWrap(
    openai_org,
    openai_api_key
)

tss = Transcripts(openai_client)

tss.gen_agent_transcripts(
    user_prompt=user_prompt_gen_transcript,
    quantity=1,
    output_dir=prj_root + 'output/t1'
)


2023-12-19 18:35:16 - openAiLogger - DEBUG - 3b78d11e-aab1-4e44-bb50-9173da0ed0ee | CreateCompletions | Input: {'user_prompt': 'The transcript should be of an executive assistant receiving calls for \ntheir executive and needing to schedule a follow-up at a later date.  Randomly decide on a date and time and how many \nturns it takes to book the appointment.  Randomly decide on people names, but make sure to use diverse names.  Don\'t \nuse the words "my executive" or "your executive" or similar expressions.  Randomly decide on the reason the executive \nis unavailable.  Don\'t use an executive service as a business.', 'sys_prompt': 'You are helping me generate example transcripts.  Do not reply back with anything other \nthan the transcript content itself.  No headers or footers.  Separate each turn with a blank line.  Each line should \nstart with either "USER: " or "AGENT: ".', 'model': 'gpt-4-1106-preview'}
2023-12-19 18:35:29 - openAiLogger - DEBUG - 3b78d11e-aab1-4e44-bb50-9173da